In [1]:
CSSOM => CSS Rules => Selector(*)
:지칭, 구조, 관계 => find*

가상선택자
#아이디, .클래스, .클래스.클래스, 태그이름#아이디, #아이디.클래스
(주의) 띄어쓰기는 자손, 자식 >, 바로다음형제 +, 다음 형제노드들 ~
:has, :is, :nth-child(같은 부모를 공유하는 자식들의 순번), :nth-of-type(같은 종류의 태그 중 순번)
           :nth-child(1) => first-child
           :nth-child(-1) => last-child

직접적용해보자. 지금은 누구나 할 수있고, 배울 수 있게 되었다.
KT 센터장님 - 해외 물건 떼어다가 파는 사람이 있었는데, 크롤링을 통해 구매대행 자동화함.
익숙해지기만 하면 많은 것들을 해볼 수 있다.
---
크롤러란
:link를 따라다니면서, 웹페이지에 무슨 내용이 있었는지 수집(index/indexing/scraping) + 다음 link를 찾아 또 따라다니는
:Tree(BFS, DFS) => link를 어떻게 찾나? => 정규화 => 다음번 방문할 link(방문한 곳, 방문할 곳에 포함되지 않는 link)
=> focused crawling(*) + 스크래핑 + DB(다음 주 월요일)
=> (수지 1,2,3,4,5,6...) 수집한 내용 중에서 우선순위를 매겨야한다. citation(href;linking) => PageRank

---
딥러닝의 역사를 알면 좋다.
세상은 강약약강, 세상은 불공평하다.
이걸 교육을 하면서도 잰 안될꺼야라고 생각한다.
심리학과에서는 사람은 왜 이행동을 할까 하고 사람에 대해 연구하는 학과이다. -> 그중 인지 심리학 -> 인지언어 심리학
교수님 VS 선생님
인지언어 심리학 -> 우리도 우리를 모른다 -> 그럼 어떻게 인공지능이 인간을 알수 있겠는가?

어제 만든 크롤러에 스택을 사용했다.
---

SyntaxError: invalid syntax (4003628311.py, line 1)

In [157]:
from requests.compat import urlparse
from requests import get
from urllib.parse import urljoin
ua = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36"

def canFetch(url, path):
    # disallowList = list()
    resp = get(urljoin(url, '/robots.txt'))
    
    if resp.status_code != 200:
        return True

    disallowList = [link for link in re.findall(r'Disallow\s*:\s*(.+)',resp.text, re.IGNORECASE | re.MULTILINE)]

    if urlparse(path).path in disallowList:
        return False

    return True

# canFetch('https://lms.sunde41.net','/')

In [158]:
from requests import request
from requests.exceptions import HTTPError
from time import sleep

def download(url, params={}, data={}, headers={}, method="GET", retries=3):
    # if not canFetch(url, url):
    #     print('수집하면 안됨')
    resp = request(method, url, params=params, data=data, headers=headers)
    try:
        resp.raise_for_status()
    except HTTPError as e:
        if 499 < resp.status_code and retries > 0:
            print('재시도 중')
            sleep(5)
            return download(url, params, data, headers, method, retries=1 )
        else: 
            print(e.response.status_code)
            print(e.request.headers)
            print(e.response.headers)
            return None
    return resp

In [179]:
import re
from bs4 import BeautifulSoup
from urllib.parse import urlparse

###### focused Crawling 전략
# 전략1. depth 제한하기
# 전략2. domain 제한하기
# 전략3. HTML Tags 제한하기
# 전략4. 만들어내기..

#           구글검색결과             depth = 1
# 1:이미지 2:블로그 3:뉴스 4:카페 5 6  depth = 2
######

URLs = []
Visited = []
Skipped = []

# 전략1. 깊이로 제한
depth = 5

# 전략2. 도메인으로 제한
optin = ['news.naver.com',"n.news.naver.com","imgnews.pstatic.net"]
optout = ['v.daum.net']

# 수정
# URLs.append({'link':'https://search.daum.net/search?q=수지','depth':0})
URLs.append({'link':'https://news.naver.com','depth':0})

NEWS_TEXT = './naver_news'
NEWS_IMG = "./naver_img"

while URLs:
    url = URLs.pop(-1) # 0:FIFO:Queue:BFS, -1:LIFO:STACK:DFS

    # 전략1. 깊이로 제한
    if url['depth'] > depth:
        Skipped.append(url['link'])  #DB를 사용하면 칼럼하나를 사용하여 표시하면 된다.
        continue

    ### 여기부터의 코드를 함수화 시키면 여러개의 스레드를 사용하여 실행시킬 수 있다.
    Visited.append(url['link'])
    
    resp = download(url['link'], headers={"user-agent":ua})
    
    if resp is None:
        continue

    # Scraping
    if re.search(r'image\/(\w+);?',resp.headers['content-type']):
        ext = re.search(r'image\/(\w+);?',resp.headers['content-type']).group(1)
        # 파일이름 고치세요
        fname = re.search(r'/(\w+\.jpg|jpeg|png)',url['link']).group(1)
        with open(NEWS_IMG+'/'+fname+'.'+ext, 'wb') as fp:
            fp.write(resp.content)

    # Crawling
    if re.search(r'text\/html', resp.headers['content-type']):
        dom = BeautifulSoup(resp.text, "html.parser")
        # 전략3. 뉴스인 경우 (HTML 요소로 제한)
        # 1. 메뉴
        linksA = dom.select('ul.Nlnb_menu_list li > a')
        # 2. 뉴스
        linksB = dom.select('[class$="_text"] > a[href]:has(> strong)')
        # 3. 뉴스 본문
        linksC = dom.select('#dic_area a[href], #dic_area img[data-src]')
        article = dom.select_one('#dic_area')

        # Scraping for Indexing
        if article:
            with open(NEWS_TEXT+'/'+re.search(r'(\d{10})$', url['link']).group(1)+'.txt','w',encoding='UTF-8') as fp:
                fp.write(article.get_text())

        for link in linksA+linksB+linksC:
            if link.has_attr('href'):
                newURL = link.attrs['href']
            if link.has_attr('src'):
                newURL = link.attrs['src']
            if link.has_attr('data-src'):
                newURL = link.attrs['data-src']
        
        # .sa|ss_text > a:has("> strong") # 브라우저 console 창에서 확인할 수이 있다.
        # jQuery 자바스크립트를 한번 감싸서 사용하기 편한기능을 제공하는 것.
        # 최근에 다시 jQuery 쓰기시작하는 추세
        
        # for link in dom.select('''a[href], frame[src], iframe[src], img[src],
        # audio[src], video[src], style[src], link[src]'''): # form['action']은?
        #     newURL = link.attrs['href'] if link.has_attr('href') else link.attrs['src']

            if not re.match(r'#|javascript|mailto|data', newURL): #data: base인코딩된 이미지데이터제외
                # urljoin(url['link'], newURL)
                normalizedURL = urljoin(url['link'], newURL) #수정

                # 도메인 제한
                if urlparse(normalizedURL).netloc not in optin:
                    continue
                # if urlparse(normalizedURL).netloc not in optout:
                #     continue
                
                if normalizedURL not in Visited and \
                   normalizedURL not in [url['link'] for url in URLs] and \
                   normalizedURL not in Skipped:
                    URLs.append({'link':normalizedURL, 'depth':url['depth']+1}) #수정
                    
    print(len(URLs), len(Visited))

13 1
12 2
11 3
10 4
9 5
8 6
7 7
55 8
55 9
54 10
54 11
53 12
54 13
53 14


AttributeError: 'NoneType' object has no attribute 'group'

In [156]:
# 디버깅 방법!
# url = "https://news.naver.com"
# url = "https://news.naver.com/section/104"
resp = get(url, headers={'user-agent':ua})
dom = BeautifulSoup(resp.text,"html.parser")
# dom.select('ul.Nlnb_menu_list li > a')
dom.select('[class$="_text"] > a[href]:has(> strong)')

# # 1. 메뉴
#         linksA = dom.select('ul.Nlnb_menu_list li > a')
#         # 2. 뉴스
#         linksB = dom.select('[class$="_text"] > a[href]:has(> strong)')
#         # 3. 뉴스 본문
#         linksC = dom.select('#dic_area a[href]')
#         article = dom.select('#dic_area')
#         print("왜 안나오징",article)

InvalidSchema: No connection adapters were found for "{'link': 'https://news.naver.com/main/opinion/home.naver', 'depth': 2}"

In [162]:
print(len(URLs),len(Visited),len(Skipped),URLs[-1],Visited[-1],Skipped[-1])

IndexError: list index out of range

In [98]:
from os import mkdir, listdir

In [122]:
listdir('.')

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


['myfirstdjango',
 'Untitled1.ipynb',
 '.DS_Store',
 '0313_http.ipynb',
 'test2.html',
 'Untitled.ipynb',
 'myvenv2',
 '\x08crawler.ipynb',
 'crawler(2).ipynb',
 '0314_http.ipynb',
 'naver_news',
 'idea.md',
 'class_material',
 'README.md',
 'img4.jpg',
 'test.html',
 '.gitignore',
 '.ipynb_checkpoints',
 '.git',
 'practice',
 'crawler(1).ipynb']

In [100]:
mkdir('./naver_news')

In [123]:
pwd

'/Users/myoungjikim/2024_Korea_iisw'

In [143]:
[url['link'] for url in URLs]

int

In [151]:
resp = get("https://imgnews.pstatic.net/image/001/2024/03/15/PYH2020111815300005600_P4_20240315082106032.jpg?type=w647")
re.search(r'image\/(\w+);?',resp.headers['content-type']).group(1)

'jpeg'

In [154]:
re.search(r'/(\w+\.jpg|jpeg|png|bmp|gif)','https://imgnews.pstatic.net/image/001/2024/03/15/PYH2020111815300005600_P4_20240315082106032.jpg?type=w647')\
.groups(1)

('PYH2020111815300005600_P4_20240315082106032.jpg',)

In [173]:
URLs

[{'link': 'https://n.news.naver.com/mnews/article/001/0014565620', 'depth': 2},
 {'link': 'https://n.news.naver.com/mnews/article/081/0003437403', 'depth': 2},
 {'link': 'https://n.news.naver.com/mnews/article/008/0005012334', 'depth': 2},
 {'link': 'https://n.news.naver.com/mnews/article/016/0002280723', 'depth': 2},
 {'link': 'https://n.news.naver.com/mnews/article/001/0014565542', 'depth': 2},
 {'link': 'https://n.news.naver.com/mnews/article/016/0002280598', 'depth': 2},
 {'link': 'https://n.news.naver.com/mnews/article/016/0002280974', 'depth': 2},
 {'link': 'https://n.news.naver.com/mnews/article/047/0002425929', 'depth': 2},
 {'link': 'https://n.news.naver.com/mnews/article/002/0002323603', 'depth': 2},
 {'link': 'https://n.news.naver.com/mnews/article/469/0000790626', 'depth': 2},
 {'link': 'https://n.news.naver.com/mnews/article/123/0002329725', 'depth': 2},
 {'link': 'https://n.news.naver.com/mnews/article/123/0002329724', 'depth': 2},
 {'link': 'https://n.news.naver.com/mnew

In [174]:
Visited

['https://news.naver.com',
 'https://news.naver.com/?viewType=pc',
 'https://news.naver.com/section/100',
 'https://news.naver.com/section/101',
 'https://news.naver.com/section/102',
 'https://news.naver.com/section/103',
 'https://news.naver.com/section/105',
 'https://news.naver.com/section/104',
 'https://news.naver.com/main/ranking/popularDay.naver',
 'https://news.naver.com/newspaper/home?viewType=pc',
 'https://news.naver.com/opinion/home',
 'https://news.naver.com/main/tv/index.naver?mid=tvh',
 'https://news.naver.com/factcheck/main',
 'https://news.naver.com/main/ombudsman/errorArticleList.naver?mid=omb',
 'https://news.naver.com/',
 'https://news.naver.com/main/ranking/popularDay.naver?mid=etc&sid1=111',
 'https://news.naver.com/newspaper/home',
 'https://news.naver.com/main/opinion/home.naver',
 'https://news.naver.com/main/factcheck/main.naver',
 'https://n.news.naver.com/mnews/article/421/0007413465',
 'https://n.news.naver.com/mnews/article/032/0003284650',
 'https://n.ne